In [1]:
!rm -rf ../logs/

In [2]:
import sys
sys.path.append('../')
from src.utils import create_env
from src.network import Net
from src.agent import Agent
from src.trainer import Trainer

In [3]:
import gym
import torch
from tensorboardX import SummaryWriter
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
num_envs = 8
vec_env, env = create_env(num_envs)
writer = SummaryWriter('../logs/')

env_pool of size 8 and test env initialized.
SubprocVecEnv worker: got KeyboardInterrupt
SubprocVecEnv worker: got KeyboardInterrupt
SubprocVecEnv worker: got KeyboardInterrupt
SubprocVecEnv worker: got KeyboardInterrupt
SubprocVecEnv worker: got KeyboardInterrupt
SubprocVecEnv worker: got KeyboardInterrupt
SubprocVecEnv worker: got KeyboardInterrupt
SubprocVecEnv worker: got KeyboardInterrupt


In [5]:
train_var = False
net = Net(train_var)
device = torch.device('cpu')
# normal and tanh does not work
distribution = 'normal'
lr = 3e-4
agent = Agent(net, device, distribution)
optimizer = torch.optim.SGD(net.parameters(), lr)

agent initialized with normal policy


In [6]:
num_tests = 1
value_loss = 'mse'
entropy = 1e-4
gamma = 0.995
gae_lambda = 0.95
norm_adv = False
use_gae = False
ppo_eps = 0.2
num_ppo_epoch = 5
ppo_batch_size = 40

trainer = Trainer(vec_env, num_envs, env, num_tests, 
                  agent, distribution, device, optimizer, value_loss, 
                  entropy, gamma, gae_lambda, norm_adv, use_gae,
                  ppo_eps, num_ppo_epoch, ppo_batch_size, 
                  writer)

trainer initialized
training parameters:
	 value_loss: mse, gamma: 0.995, entropy_reg: 0.0001
	 use_gae: False, gae_lambda: 0.95, normalize_adv: False
	 ppo_batch_size: 40, ppo_epochs: 5


In [7]:
trainer.train(99, 150, 20)

training start
num_epochs: 99, steps_per_epochs: 150, env_steps: 20


epoch_1:   0%|                                          | 0/150 [00:00<?, ?it/s]

torch.Size([160, 3])
torch.Size([40, 3])
torch.Size([40, 3])
torch.Size([40, 3])
torch.Size([40, 3])
torch.Size([40, 3])


epoch_1:   1%|▏                                 | 1/150 [00:02<05:31,  2.22s/it]

torch.Size([160, 3])
torch.Size([40, 3])
torch.Size([40, 3])
torch.Size([40, 3])
torch.Size([40, 3])
torch.Size([40, 3])


...

epoch_1:   3%|▉                                 | 4/150 [00:08<05:25,  2.23s/it]

torch.Size([160, 3])
torch.Size([40, 3])
torch.Size([40, 3])
torch.Size([40, 3])


epoch_1:   3%|█▏                                | 5/150 [00:11<05:32,  2.29s/it]

torch.Size([40, 3])
torch.Size([40, 3])
torch.Size([160, 3])
torch.Size([40, 3])
torch.Size([40, 3])
torch.Size([40, 3])
torch.Size([40, 3])
Traceback (most recent call last):
  File "/Users/Dmitry/PycharmProjects/CarRacing/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3267, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-9589623a7721>", line 1, in <module>
    trainer.train(99, 150, 20)
  File "../src/trainer.py", line 212, in train
    policy_loss, value_loss, entropy = self.train_on_batch(batch)
  File "../src/trainer.py", line 162, in train_on_batch
    epoch_policy_loss, epoch_value_loss, epoch_entropy = self.ppo_epoch(*ppo_batch)
  File "../src/trainer.py", line 194, in ppo_epoch
    loss.backward()
  File "/Users/Dmitry/PycharmProjects/CarRacing/venv/lib/python3.6/site-packages/torch/tensor.py", line 93, in backward
    torch.autograd.backward(self, gradient, retain_graph, create_graph)
  File "/Users/Dmitry/P

KeyboardInterrupt: 

In [ ]:
obs = env.reset()

In [ ]:
agent.act([obs])